immoeliza scraping project by Majid

importing libruaries:

In [626]:
import requests
from bs4 import BeautifulSoup  

from lxml import html

import csv

primary target URLs:

In [627]:

# immoweb home page / English 
immoweb_link = 'https://www.immoweb.be/en'   
print(immoweb_link)

# a link for testing
one_page_link_test = 'https://www.immoweb.be/en/classified/apartment/for-sale/tienen/3300/20230670'
print(one_page_link_test)

# immoweb house fore sale's page
immoweb_sale_link = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=1&orderBy=relevance'
print(immoweb_sale_link)


https://www.immoweb.be/en
https://www.immoweb.be/en/classified/apartment/for-sale/tienen/3300/20230670
https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=1&orderBy=relevance


jumping from Cloudflare, cookies and othe walls

In [628]:
# it use to manage cookies
session = requests.Session()    

# for fixing  error 403
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}


In [629]:
# to extract the links from group pages
response_group_page = session.get(immoweb_sale_link, headers=headers)

soup = BeautifulSoup(response_group_page.text, 'html.parser')

if response_group_page.status_code == 200:
    print("well done for group page") 
    #print(soup.prettify()) 
else:
    print(response_group_page.status_code)

# --------------------------------------------------------------

# to extract featurs from any pages
response_one_page = session.get(one_page_link_test, headers=headers)

soup = BeautifulSoup(response_one_page.text, 'html.parser')

if response_one_page.status_code == 200:
    print("well done for one page") 
    #print(soup.prettify()) 
else:
    print(response_one_page.status_code)
#print(soup.find('title'))



well done for group page
well done for one page


extracting some features from one link:

In [630]:
# taking 5 last part of any link has some features to use
last_5_parts = one_page_link_test.split('/')[-5:]    
#print(f'Last 5 parts of the link: {last_5_parts}')


#  1. Property ID
print(f'1. Property ID: {last_5_parts[4]}')
#property_id_div = soup.find('div', 'classified__header--immoweb-code')   # Find the div containing the property ID
#property_id = (property_id_div).text.strip().split(': ')[-1]             # Extract the text and split it to get the ID
#print(f'1.property ID :  {property_id}')

#  2. Locality name
print(f'2. Locality name: {last_5_parts[2]}')

#  3. Postal code
print(f'3. Postal code: {last_5_parts[3]}')

#  4. Price
price_p = soup.find('p', 'classified__price')                         # extracted from beginning of website, received error from Financial
price = price_p.find('span', {'aria-hidden': 'true'}).text.strip()    # Extract the price from the first span
print(f'4. price :  {price}')  

#  5. Type of property (house or apartment)
print(f'5. Type of property: {last_5_parts[0]}')

#  6. Subtype of property (bungalow, chalet, mansion, ...)

#  7. Type of sale (note: exclude life sales)

#  8. Number of rooms  //////// 14

#  9. Living area (area in m²)  //////// 8 

#  10. Equipped kitchen (0/1)   

#  11. Furnished (0/1)

#  12. Open fire (0/1)

#  13. Terrace (area in m² or null if no terrace)  //////// 17 

# 14. Garden (area in m² or null if no garden)

# 15. Number of facades  //////// 6

# 16. Swimming pool (0/1)

#17. State of building (new, to be renovated, ...)  //////// 5





1. Property ID: 20230670
2. Locality name: tienen
3. Postal code: 3300
4. price :  €189,000
5. Type of property: apartment


In [631]:
extracted_data = []

rows = soup.find_all('tr')          # finding all rows in the table

for row in rows:                    # extracting header and data in allrows
    header_part = row.find('th')
    data_part =   row.find('td')
    
    if header_part and data_part:   # checking, then cleaning
        header_part_cleaned = ''.join([str(item).strip() for item in header_part.contents if isinstance(item, str)])
        data_part_cleaned =   ''.join([str(item).strip() for item in data_part.contents   if isinstance(item, str)])
                
        extracted_data.append((header_part_cleaned, data_part_cleaned))  # adding the cleaned extracted texts to list

extracted_data

#for header, data in extracted_data:
    #print(f"{header_part} {data_part}")

#print(extracted_data[7])   # living area size

[('Available as of', 'After signing the deed'),
 ('Construction year', '1972'),
 ('Floor', '1'),
 ('Number of floors', '10'),
 ('Building condition', 'To renovate'),
 ('Number of frontages', '4'),
 ('Asbestos certificate is available', 'Yes'),
 ('Living area', '81'),
 ('Living room surface', '27'),
 ('Kitchen surface', '7'),
 ('Bedrooms', '2'),
 ('Bedroom 1 surface', '16'),
 ('Bedroom 2 surface', '8'),
 ('Bathrooms', '1'),
 ('Toilets', '1'),
 ('Basement', 'Yes'),
 ('Terrace surface', '22'),
 ('Elevator', 'Yes'),
 ('TV cable', 'Yes'),
 ('Primary energy consumption', '202'),
 ('Energy class', 'C'),
 ('Reference number of the EPC report', '0003309605'),
 ('CO₂ emission', 'Not specified'),
 ('Yearly theoretical total energy consumption', 'Not specified'),
 ('As built plan', 'No'),
 ('Conformity certification for fuel tanks', 'Not specified'),
 ('Heating type', 'Gas'),
 ('Double glazing', 'Yes'),
 ('Flood zone type', 'Non flood zone'),
 ('Price', ''),
 ('Cadastral income', ''),
 ('Tenement 

generating the links of 333 pages 

In [632]:
first_group_pages_link_sale = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2'
p = 1
last_p = 333
group_pages_link_sale = []

# adding 1st page in 1st cell of list
group_pages_link_sale.append(first_group_pages_link_sale)

for p in range(2,last_p+1):  
    group_pages_link_sale.append(first_group_pages_link_sale +"&page=" + str(p))   # adding the link + new number of pages

group_pages_link_sale

['https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=2',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=3',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=4',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=5',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=6',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=7',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=8',
 'https://www.immoweb.b

extracting all links of itemes for sale from 1 page:

In [633]:

# finding all links in tag <a>  with class="card__title-link" and with href 
all_links_per_page = []
for tag_a in soup.find_all('a', class_="card__title-link", href=True):
    all_links_per_page.append(tag_a['href'])

all_links_per_page


[]

filter the "new real estate

test

In [634]:
group_pages_link_sale_test = group_pages_link_sale[:10]
group_pages_link_sale_test




['https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=2',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=3',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=4',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=5',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=6',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=7',
 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&amp%3BorderBy=relevance&amp%3Bpage=2&page=8',
 'https://www.immoweb.b

extracting all linkes (+10000 links) for sale from immoweb:

In [635]:
all_links = []              

#group_pages_link_sale_test = group_pages_link_sale[:2]

for l in group_pages_link_sale:#_test:          # a loop to scrape that 333 group pages of immoweb

    session = requests.Session()               
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = session.get(l, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for tag_a in soup.find_all('a', class_="card__title-link", href=True):   # extracting all item links paer group page
        all_links.append(tag_a['href'])


#all_links               


KeyboardInterrupt: 

save all the links as a csv file:

In [223]:
with open("all_linkes.csv",mode="w",newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(all_links)  


(to test) extracting 2 item frpm all links

In [602]:
all_links_test = all_links [:1000]

data_set_immoweb_test = []

for counter in range(len(all_links_test)):

    counter_row = all_links_test[counter]
    #print(counter_row)
    session = requests.Session()               
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = session.get(counter_row, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')


    # property_id_div = soup.find('div', 'classified__header--immoweb-code')   # Find the div containing the property ID
    # property_id = (property_id_div).text.strip().split(': ')[-1]             # Extract the text and split it to get the ID
    #print(f'1.property ID :  {property_id}')

    #  4. Price
    price_p = soup.find('p', 'classified__price')                         
    price = price_p.find('span', {'aria-hidden': 'true'}).text.strip()    
    #print(f'4.price :  {price}')  
    #data_set_immoweb_test [counter,0] = counter
    #data_set_immoweb_test [counter,1] = price
    data_set_immoweb_test.append([counter_row, price])#property_id, price])
    

data_set_immoweb_test
    

AttributeError: 'NoneType' object has no attribute 'find'

to test final code

In [ ]:
with open("all_data.csv",mode="w",newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(all_links) 